# Self-driving car Nanodegree - Term 1

## Project 3: **Advanced Lane Lines** 
***
In this project, We attempted to identify the lane boundaries in a video.

---
#### Importing packages

In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import pickle
import numpy as np
import glob

#### Helper functions
---
##### Apply a distortion correction to raw images.

In [ ]:
def grayscale(image, order='BGR'):
    if order == 'BGR':
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif order == 'RGB':
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray

def finding_corner(images, obj_points , img_points):
    for idx, fname in enumerte(images):
        img = cv2.imread(fname)
        gray = grayscale(img, cv2.COLOR_BGR2GRAY)
        # using 9x6 instead of 8x6
        ret, corners = cv2.findChessboardCorners(gray,(9,6), None)
        if ret == True:
            obj_points.append(objp)
            img_points.append(corners)
    return obj_points, img_points

def undistort_image(image, obj_points, img_points):           
    img = cv2.imread(image)
    img_size = (img.shape[1],img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points,img_size, None, None)
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    return undistorted

---
##### Use color transforms, gradients, etc., to create a thresholded binary image.

In [7]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return binary_output

def magnitude_threshold(img, sobel_kernel=3, mag_thresh=(0, 255)): 
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def direction_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(grApply a perspective transform to rectify binary image ("birds-eye view").ay, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output   

---
##### Apply a perspective transform to rectify binary image ("birds-eye view").

In [8]:
def corners_unwarp(image, corners, offset):
    img_size = (image.shape[1], imageshape[0])
    src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
    dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                 [img_size[0]-offset, img_size[1]-offset], 
                                 [offset, img_size[1]-offset]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(undist, M, img_size)
    return warped, M

In [9]:
# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output


---
#### Main Function

In [ ]:
'''Directory'''
TEST_IMG_FOLDER = '../test_images/'
STRAIGHTLINE_IMG = '../test_images/straight_lines2.jpg'
CAMERA_IMG_FOLDER = '../camera_cal/'
OUT_IMG_FOLDER = '../output_images/'

'''Define Parameters'''
KERNEL_SIZE = 3
OFFSET = 
NX = 9
NY = 6

#Prepare obj poinrts
objp = np.zeros((NX * NY, 3), np.float32)
objp[:,:2] = np.mgrid[0:NX, 0:NY].T.reshape(-1,2)
obj_points = [] #3d points in real world space
img_points = [] #2d points in image points

images = glob.glob(CAMERA_IMG_FOLDER + '.jpg')
finding_corner(images, obj_points , img_points):
    
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray_img = grayscale(img, order ='BRG'
    undistort = cal_undistort(gray_img, objp, img_points)        
    gradx = abs_sobel_thresh(undistort, orient='x', sobel_kernel= KERNEL_SIZE, thresh=(0, 255))
    grady = abs_sobel_thresh(undistort, orient='y', sobel_kernel= KERNEL_SIZE, thresh=(0, 255))
    mag_binary = mag_thresh(undistort, sobel_kernel= KERNEL_SIZE, mag_thresh=(0, 255))
    dir_binary = dir_threshold(undistort, sobel_kernel= KERNEL_SIZE, thresh=(0, np.pi/2))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
        # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()


# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "calibration_wide/wide_dist_pickle.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.tight_layout()

ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()